We used Python version 3.11.5.

# 0. Installing requirements, loading packages & modules

In [1]:
# creating virtual environment
!python -m venv ENV 

#!pip list
# install module requirements
!pip install -r requirements.txt

# loading packages 
import csv
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as WBW
from selenium.webdriver.support import expected_conditions as EC

# Loading module with scraping functions
from scraper import *

  Using cached selenium-2.16.0.tar.gz (3.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      no previously-included directories found matching '*'
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\leona\AppData\Local\Temp\pip-install-gv6zamw2\selenium_6ae39494c5844175ad756e4b8e688033\setup.py", line 55, in <module>
          src_root = setup_python3()
                     ^^^^^^^^^^^^^^^
        File "C:\Users\leona\AppData\Local\Temp\pip-install-gv6zamw2\selenium_6ae39494c5844175ad756e4b8e688033\setup.py", line 47, in setup_python3
          util.run_2to3(outfiles_2to3)
          ^^^^^^^^^^^^^
      AttributeError: module 'distutils.util' has no attribute 'run_2to3'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× En

# 1. Scraping URLS of Phone Products

#### DatArt

In [2]:
# 1 Finding the basic URL that can be extended to obtain the links of certain products
DatArt_basic_url = "https://www.datart.cz"

# 2 Finding category website with all the products listed (here by manually setting the limit of products shown manually in the browser)
DataArt_category_url = "https://www.datart.cz/mobilni-telefony.html?limit=1100" 

product_urls_DatArt = get_product_urls_DatArt(DatArt_basic_url, DataArt_category_url)

964 products found.(2024-01-28 22:58:54)


#### CZC

In [3]:
# 1 Finding the basic URL that can be extended to obtain the links of certain products
CZC_basic_url = "https://www.czc.cz"

# 2 Finding category website with all the products listed (here no manual limit for the number of products shown can be set)
CZC_category_url = "https://www.czc.cz/mobilni-telefony/produkty"

# 3 Instead all pages of the category url have to be individually scraped
CZC_category_urls = [CZC_category_url] # category_url is added as it is the first page with products

# 4 Generating URL for all pages of the category website

# Create list of page limits
page_limits = list(range(27, 1000, 27)) 

# Iterate over the pages and construct URLs for every page of the category website
for page_limit in page_limits:
    url_with_page_limit = f"{CZC_category_url}?q-first={page_limit}"
    CZC_category_urls.append(url_with_page_limit)

product_urls_CZC = get_product_urls_CZC(CZC_basic_url, CZC_category_urls)

Page 1 complete
Page 2 complete
Page 3 complete
Page 4 complete
Page 5 complete
Page 6 complete
Page 7 complete
Page 8 complete
Page 9 complete
Page 10 complete
Page 11 complete
Page 12 complete
Page 13 complete
Page 14 complete
Page 15 complete
Page 16 complete
Page 17 complete
Page 18 complete
Page 19 complete
Page 20 complete
Page 21 complete
Page 22 complete
Page 23 complete
Page 24 complete
Page 25 complete
Page 26 complete
Page 27 complete
Page 28 complete
Page 29 complete
Page 30 complete
Page 31 complete
Page 32 complete
Page 33 complete
Page 34 complete
Page 35 complete
Page 36 complete
Page 37 complete
Page 38 complete
857 products found.(2024-01-24 12:04:57)


#### Electroworld

In [3]:
# 1 Finding the basic URL that can be extended to obtain the links of certain products
Electroworld_basic_url = "https://www.electroworld.cz"

# 2 Finding category page with all the products listed (here no manual limit for the number of products shown can be set)
Electroworld_category_url =  "https://www.electroworld.cz/chytre-mobily/sort-by_mostExpensive" # sorted by most expensive gives unique products for each page

# 3 Instead all pages of the category url have to be individually scraped
Electroworld_category_urls = [Electroworld_category_url] # category_url is added as it is the first page with products

# 4 Generating URL for all pages of the category website

# Create list of pages 
pages = list(range(2, 43, 1)) 

# Iterate over the pages and construct URLs for every page of the category website
for page in pages:
    url_with_page = f"{Electroworld_category_url}?page={page}"
    Electroworld_category_urls.append(url_with_page)

product_urls_Electroworld = get_product_urls_Electroworld(Electroworld_basic_url, Electroworld_category_urls) 

Page 1 complete
Page 2 complete
Page 3 complete
Page 4 complete
Page 5 complete
Page 6 complete
Page 7 complete
Page 8 complete
Page 9 complete
Page 10 complete
Page 11 complete
Page 12 complete
Page 13 complete
Page 14 complete
Page 15 complete
Page 16 complete
Page 17 complete
Page 18 complete
Page 19 complete
Page 20 complete
Page 21 complete
Page 22 complete
Page 23 complete
Page 24 complete
Page 25 complete
Page 26 complete
Page 27 complete
Page 28 complete
Page 29 complete
Page 30 complete
Page 31 complete
Page 32 complete
Page 33 complete
Page 34 complete
Page 35 complete
Page 36 complete
Page 37 complete
Page 38 complete
Page 39 complete
Page 40 complete
Page 41 complete
Page 42 complete
741 products found.(2024-01-28 23:00:31)


Improvements:
- For CZC code: Automatic handling of pages, instead of manually setting the project number limit of 1000 above. 27 can stay if necessary (or trying out numbers 1:50 until data is obtained)
- Electroworld: (.)

# 2. Scraping Product Information 

#### DatArt

For all products (product URL's) found, the product information is scraped. For every characteristic, a function is defined in scraper.py. For some variables like wireless technologies, lists of characteristics are scraped. For each of these characteristics a 1-values is assigned to show that these characteristic is available.

In [13]:
product_urls_DatArt[189]

'https://www.datart.cz/Mobilni-telefon-Aligator-A321-Senior-seda-cerna.html'

In [5]:
products_DatArt =[]
for product_number, product_url in enumerate(product_urls_DatArt, start = 1):

    # Getting BeautifulSoup object of product (URL)
    page = requests.get(product_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Placeholder for product characteristics
    product_entry_DatArt = {} 
    product_entry_DatArt["ID"] = product_number
    product_entry_DatArt["online-retailer"] = "DatArt"
    product_entry_DatArt['title'] = get_product_title_DatArt(soup)
    product_entry_DatArt['price'] = get_product_price_DatArt(soup)
    product_entry_DatArt['rating'] = get_product_rating_DatArt(soup)
    product_entry_DatArt['number of ratings'] = get_product_no_ratings_DatArt(soup)
    product_entry_DatArt['display size'] = get_product_display_size_DatArt(soup)
    product_entry_DatArt['resolution width'] = get_product_resolution_w_DatArt(soup)
    product_entry_DatArt['resolution height'] = get_product_resolution_h_DatArt(soup)
    product_entry_DatArt['resolution total'] = get_product_resolution_tot_DatArt(soup)
    product_entry_DatArt['display refresh rate'] = get_product_display_refresh_rate_DatArt(soup)
    product_entry_DatArt['cutout shape'] = get_product_cutout_shape_DatArt(soup)
    product_entry_DatArt['processor manufacturer'] = get_product_processor_manufacturer_DatArt(soup)
    product_entry_DatArt['processor model'] = get_product_processor_model_DatArt(soup)
    product_entry_DatArt['number of cores'] = get_product_no_cores_DatArt(soup)
    product_entry_DatArt['processor frequency'] = get_product_processor_freq_DatArt(soup)
    product_entry_DatArt['SIM card type'] = get_product_SIM_card_type_DatArt(soup)
    product_entry_DatArt['configuration cards'] = get_product_config_cards_DatArt(soup)
    product_entry_DatArt['degree of protection'] = get_product_degree_of_protection_DatArt(soup)
    product_entry_DatArt['operating system'] = get_product_OS_DatArt(soup)
    product_entry_DatArt['system superstructure'] = get_product_system_superstructure_DatArt(soup)
    product_entry_DatArt['notification diode'] = get_product_notification_diode_DatArt(soup)
    product_entry_DatArt['internal memory'] = get_product_int_memory_DatArt(soup)
    product_entry_DatArt['RAM'] = get_product_RAM_DatArt(soup)
    product_entry_DatArt['memory card slot'] = get_product_memory_card_slot_DatArt(soup)
    product_entry_DatArt['maximum memory card size'] = get_product_max_memory_card_size_DatArt(soup)

    if get_product_wireless_tech_list_DatArt(soup):
        for obj in get_product_wireless_tech_list_DatArt(soup):
            product_entry_DatArt[obj] = 1

    product_entry_DatArt['number of rear camera lenses'] = get_product_no_rear_cam_lenses_DatArt(soup)
    product_entry_DatArt['number of front camera lense'] = get_product_no_front_cam_lenses_DatArt(soup)
    product_entry_DatArt['rear cam resolution'] = get_product_rear_cam_resolution_DatArt(soup)
    product_entry_DatArt['front cam resolution'] = get_product_front_cam_resolution_DatArt(soup)

    if get_product_camera_features_DatArt(soup):
        for obj in get_product_camera_features_DatArt(soup):
            product_entry_DatArt[obj] = 1

    product_entry_DatArt['battery type'] = get_product_battery_type_DatArt(soup)
    product_entry_DatArt['battery capacity'] = get_product_battery_capacity_DatArt(soup)

    if get_product_battery_features_DatArt(soup):
        for obj in get_product_battery_features_DatArt(soup):
            product_entry_DatArt[obj] = 1

    product_entry_DatArt['charging power'] = get_product_charging_power_DatArt(soup)

    if get_product_security_DatArt(soup):
        for obj in get_product_security_DatArt(soup):
            product_entry_DatArt[obj] = 1
    
    product_entry_DatArt['connector'] = get_product_connector_DatArt(soup)
    product_entry_DatArt['3.5mm jack'] = get_product_3_5mm_jack_DatArt(soup)
    product_entry_DatArt['warranty'] = get_product_warranty_DatArt(soup)
    product_entry_DatArt['FM-radio'] = get_product_fm_radio_DatArt(soup)
    product_entry_DatArt['colour'] = get_product_colour_DatArt(soup)
    product_entry_DatArt['brand'] = get_product_brand_DatArt(soup)
    product_entry_DatArt['width'] = get_product_width_DatArt(soup)
    product_entry_DatArt['length'] = get_product_length_DatArt(soup)
    product_entry_DatArt['depth'] = get_product_depth_DatArt(soup)
    product_entry_DatArt['volume'] = get_product_volume_DatArt(soup)
    product_entry_DatArt['weight'] = get_product_weight_DatArt(soup)

    products_DatArt.append(product_entry_DatArt)
    print(f"Product {product_number} finished")

Product 1 finished
Product 2 finished
Product 3 finished
Product 4 finished
Product 5 finished
Product 6 finished
Product 7 finished
Product 8 finished
Product 9 finished
Product 10 finished
Product 11 finished
Product 12 finished
Product 13 finished
Product 14 finished
Product 15 finished
Product 16 finished
Product 17 finished
Product 18 finished
Product 19 finished
Product 20 finished
Product 21 finished
Product 22 finished
Product 23 finished
Product 24 finished
Product 25 finished
Product 26 finished
Product 27 finished
Product 28 finished
Product 29 finished
Product 30 finished
Product 31 finished
Product 32 finished
Product 33 finished
Product 34 finished
Product 35 finished
Product 36 finished
Product 37 finished
Product 38 finished
Product 39 finished
Product 40 finished
Product 41 finished
Product 42 finished
Product 43 finished
Product 44 finished
Product 45 finished
Product 46 finished
Product 47 finished
Product 48 finished
Product 49 finished
Product 50 finished
Product 5

Converting the list of dictionaries into a json file "product_info_DatArt.json".

In [ ]:
# convert to JSON object and safe as JSON file
with open('product_info_DatArt.json', 'w') as file:

    json.dump(products_DatArt, file)

#### Electroworld 

For the products (product URL's) from Electroworld, only the BeatifulSoup object is not enough to scrape the product characteristics. Here, a large part of the product characteristics is hidden behind a button. Therefore, the selenium module is used to click on the button in the background and scrape the data from the resulting dynamic webpage.

In [ ]:
# Next Steps:
# 1. Making loop for EW
# 2. Running loop for EW and DatArt for 50 products each
# After creating two big datasets for DatArt and EW:
    	# 1. Decide about SIM card variable: 
                # - DatArt -> 
                # - Electroworld -> SIM card type: "nano sim", configuration cards: "Single SIM + eSIM"
        # 2. Decide about EW bluetooth variable: Changing it to 0/1 dummy?
        # 3. Decide about EW changing wireless charging power to charging power and include it via try except clause (as wireless charging is included as a variable anyway)
        # 4. DatArt: Making memory card slot to dummy?
# 3. Adjust all the mistakes that can be found from this
# 4. Running loop again to get all products data for json files
# 5. Open new file for summary statistics and analysis
# 6. Document all variables used in the README file; Make a table: One column for one online-retailer (all variables first which are available for every retailer)

Checking the following variables:
- EW: charging power
- SIM card type: 1. Setting same variable names. 2. Changing "eSIM, nano sim" to "nano SIM + eSIM" and "nano sim" to "nano SIM". "configuration cards" should be only available for Electroworld.
- DatArt: Renaming "maximum memory card size" to "memory card slot". Set it to 0 if "Doesn't support..." or to 1 if its a number."
- DatArt: checking the product with "Other colour"
- EW: Changing depth, width and height in their scaling; probably divide the values and round them
- EW: changing wireless charging performance to charging power?
- DatArt: Equivalent to "in-display fingerprint reader" in DatArt?
- EW: Check if RAM is correct, (.) for EW as there are many "None" values

In [4]:
products_Electroworld =[]
for product_number, product_url in enumerate(product_urls_Electroworld[0:49], start = 1):

    # Start the browser
    #driver = webdriver.Chrome()

    # Navigate to the website
    #driver.get(product_url)

    product_entry_Electroworld = {} 
    # A large part of the product characteristics is hidden behind a button to extend the parameters shown on the product page
    # Here the selenium package is used to push the button of the webpage in the background to scrape the hidden product information
    try:
        soup = get_soup_Electroworld(product_url)
    except Exception as e:
        print(f"Failed to retrieve data. Final Exception: {str(e)}")

    product_entry_Electroworld["ID"] = product_number
    product_entry_Electroworld["online-retailer"] = "Electroworld"
    product_entry_Electroworld["title"] = get_product_title_Electroworld(soup)
    product_entry_Electroworld["price"] = get_product_price_Electroworld(soup)
    product_entry_Electroworld["colour"] = get_product_colour_Electroworld(soup)
    product_entry_Electroworld["RAM"] = get_product_RAM_Electroworld(soup)
    product_entry_Electroworld["width"] = get_product_width_Electroworld(soup)
    product_entry_Electroworld["depth"] = get_product_depth_Electroworld(soup)
    product_entry_Electroworld["length"] = get_product_length_Electroworld(soup)
    product_entry_Electroworld["volume"] = get_product_volume_Electroworld(soup)
    product_entry_Electroworld["weight"] = get_product_weight_Electroworld(soup)
    product_entry_Electroworld["bluetooth"] = get_product_bluetooth_Electroworld(soup)
    product_entry_Electroworld["WiFi standard"] = get_product_wifi_standard_Electroworld(soup)
    product_entry_Electroworld["NFC"] = get_product_NFC_Electroworld(soup)
    product_entry_Electroworld["connector"] = get_product_connector_Electroworld(soup)
    product_entry_Electroworld["3.5mm jack"] = get_product_3_5_mm_jack_Electroworld(soup)
    product_entry_Electroworld["display size"] = get_product_display_size_Electroworld(soup)
    product_entry_Electroworld["display type"] = get_product_display_type_Electroworld(soup)
    product_entry_Electroworld["resolution width"] = get_product_display_resolution_w_Electroworld(soup)
    product_entry_Electroworld["resolution height"] = get_product_display_resolution_h_Electroworld(soup)
    product_entry_Electroworld["resolution total"] = get_product_display_resolution_tot_Electroworld(soup)
    product_entry_Electroworld["display refresh rate"] = get_product_display_refresh_rate_Electroworld(soup)
    product_entry_Electroworld["resolution label"] = get_product_display_resolution_label_Electroworld(soup)
    product_entry_Electroworld["display fineness"] = get_product_display_fineness_Electroworld(soup)
    product_entry_Electroworld["brand"] = get_product_brand_Electroworld(soup)
    product_entry_Electroworld["smart"] = get_product_smart_Electroworld(soup)
    product_entry_Electroworld["water resistant"] = get_product_water_resistant_Electroworld(soup)
    product_entry_Electroworld["degree of protection"] = get_product_degree_of_protection_Electroworld(soup)
    product_entry_Electroworld["operating system"] = get_product_operating_system_Electroworld(soup)
    product_entry_Electroworld["os brand"] = get_product_os_brand_Electroworld(soup)
    product_entry_Electroworld["GPS"] = get_product_gps_Electroworld(soup)
    product_entry_Electroworld["wireless charging"] = get_product_wireless_charging_Electroworld(soup)
    product_entry_Electroworld["fast charging"] = get_product_fast_charging_Electroworld(soup)

    if get_product_security_controls_Electroworld(soup):
        for obj in get_product_security_controls_Electroworld(soup):
            product_entry_Electroworld[obj] = 1

    product_entry_Electroworld["processor manufacturer"] = get_product_processor_manufacturer_Electroworld(soup)
    product_entry_Electroworld["processor model"] = get_product_processor_model_Electroworld(soup)
    product_entry_Electroworld["number of cores"] = get_product_number_of_cores_Electroworld(soup)
    product_entry_Electroworld["4G/LTE"] = get_product_4G_LTE_Electroworld(soup)
    product_entry_Electroworld["dual SIM support"] = get_product_dual_sim_support_Electroworld(soup)
    product_entry_Electroworld["sim card type"] = get_product_sim_card_type_Electroworld(soup)
    product_entry_Electroworld["5G"] = get_product_5G_Electroworld(soup)
    product_entry_Electroworld["configuration cards"] = get_product_config_cards_Electroworld(soup)
    product_entry_Electroworld["memory card slot"] = get_product_memory_card_slot_Electroworld(soup)
    product_entry_Electroworld["battery capacity"] = get_product_battery_capacity_Electroworld(soup)
    product_entry_Electroworld["charging power"] = get_product_charging_power_Electroworld(soup)
    product_entry_Electroworld["internal memory"] = get_product_internal_memory_Electroworld(soup)
    product_entry_Electroworld["front cam"] = get_product_front_cam_Electroworld(soup)
    product_entry_Electroworld["rear cam resolution"] = get_product_rear_cam_resolution_Electroworld(soup)
    product_entry_Electroworld["front cam resolution"] = get_product_front_cam_resolution_Electroworld(soup)
    product_entry_Electroworld["number of rear camera lenses"] =  get_product_no_rear_cam_lenses_Electroworld(soup)
    product_entry_Electroworld["number of front camera lenses"] = get_product_no_front_cam_lenses_Electroworld(soup)
    product_entry_Electroworld["FM-radio"] = get_product_fm_radio_Electroworld(soup)
    product_entry_Electroworld["built-in flash"] = get_product_built_in_flash_Electroworld(soup)
    product_entry_Electroworld["flash type"] = get_product_flash_type_Electroworld(soup)
    product_entry_Electroworld["wireless charging performance"] = get_product_wireless_charging_performance_Electroworld(soup)
    product_entry_Electroworld["number of ratings"] = get_product_no_ratings_Electroworld(soup)
    product_entry_Electroworld["rating"] = get_product_rating_Electroworld(soup)
        
    products_Electroworld.append(product_entry_Electroworld)
    print(f"Product {product_number} finished")

Product 1 finished
Product 2 finished
Product 3 finished
Product 4 finished
Product 5 finished
Product 6 finished
Attempt 1 failed. Exception: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7690D2142+3514994]
	(No symbol) [0x00007FF768CF0CE2]
	(No symbol) [0x00007FF768B976AA]
	(No symbol) [0x00007FF768BE1860]
	(No symbol) [0x00007FF768BE197C]
	(No symbol) [0x00007FF768C24EE7]
	(No symbol) [0x00007FF768C0602F]
	(No symbol) [0x00007FF768C228F6]
	(No symbol) [0x00007FF768C05D93]
	(No symbol) [0x00007FF768BD4BDC]
	(No symbol) [0x00007FF768BD5C64]
	GetHandleVerifier [0x00007FF7690FE16B+3695259]
	GetHandleVerifier [0x00007FF769156737+4057191]
	GetHandleVerifier [0x00007FF76914E4E3+4023827]
	GetHandleVerifier [0x00007FF768E204F9+689705]
	(No symbol) [0x00007FF768CFC048]
	(No symbol) [0x00007FF768CF8044]
	(No symbol) [0x00007FF768CF81C9]
	(No symbol) [0x00007FF768CE88C4]
	BaseThreadInitThunk [0x00007FFBC117257D+29]
	RtlUserThreadStart [0x00007FFBC286AA58+40]

Product 7 finished
Product 8 f

Converting the list of dictionaries into a json file "product_info_Electroworld.json".

In [5]:
# convert to JSON object and safe as JSON file
with open('product_info_Electroworld.json', 'w') as file:

    json.dump(products_Electroworld, file)

Creating CSV file for a different representation in the working directory.

##### DatArt

In [54]:
# Specify the paths for CSV files
csv_file_path = 'Product_Table_DatArt.csv'

# Open the JSON file and load its content
with open('product_info_DatArt.json', 'r') as json_file:
    Products_Data_DatArt = json.load(json_file)
    
# Extract all unique keys from the JSON data
all_keys = set()
for entry in Products_Data_DatArt:
    all_keys.update(entry.keys())

# Move "ID" to the front of the list of keys
all_keys = ['ID'] + list(all_keys - {'ID'})

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(all_keys)

    # Write the data
    for entry in Products_Data_DatArt:
        # Create a row with "ID" as the first column
        row_data = [entry.get('ID', '')] + [entry.get(key, '') for key in all_keys[1:]]
        csv_writer.writerow(row_data)

##### Electroworld

In [56]:
# Specify the paths for CSV files
csv_file_path = 'Product_Table_Electroworld.csv'

# Open the JSON file and load its content
with open('product_info_Electroworld.json', 'r') as json_file:
    Products_Data_Electroworld = json.load(json_file)
    
# Extract all unique keys from the JSON data
all_keys = set()
for entry in Products_Data_Electroworld:
    all_keys.update(entry.keys())

# Move "ID" to the front of the list of keys
all_keys = ['ID'] + list(all_keys - {'ID'})

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(all_keys)

    # Write the data
    for entry in Products_Data_Electroworld:
        # Create a row with "ID" as the first column
        row_data = [entry.get('ID', '')] + [entry.get(key, '') for key in all_keys[1:]]
        csv_writer.writerow(row_data)

#### 3. Ideas for Analysis

Steps:
0) Which data can we use legally without problems? Ebay, DataArt etc.

1) Creating a function to save all necessary data for one product simultaneously.
2) Create a function to let the function from 1) run over all product URL's.
2) (How to save the data of the product description -> we need to look at different products and see what variables we can define.)
3) How to work with github, commits, pull etc.?
4) How to create a nice table in github to document aims and checkpoints.
5) How to structure the repository -> documents for functions, loading packages, ReadMe etc.
6) Running scripts on christmas again, to see if their is a price drop because of price offers
7) Starting data analysis -> simple averages, counts etc. -> can we create a dashboard on github? (Share or products etc.)
8) Further Statistical Analysis - Ideas:
    1) Predicing prices for new products (price modelling), finding out which variables drive prices the most (factor)-> Does it makes sense to do a price model as prices consist largely of the purchase price; could we find those purchase prices somewhere and join them with our data?
    2) Brand Analysis: Explore the average prices of smartphones for each brand. Analyze market share based on the number of products or total sales for each brand.
    3) Storage Impact: Investigate how storage capacity (GB) affects smartphone prices. Explore the most popular storage capacity among consumers.
    4) Segment the market based on different characteristics (e.g., high-end, mid-range, budget). One could then do 1) on all different segments to explore differences.
    5) Consumer Preferences: Survey or analyze customer reviews to understand what features users value the most. Identify common positive and negative sentiments related to smartphone characteristics.
    6) Profitability Analysis (given that we know production or purchase prices): Estimate profit margins for different smartphones based on their production costs and prices.
Identify the most profitable products in your dataset. One could use the number of ratings as a proxy for the number of sales.


8) Which data tests and checks can we include?

How we could build a price model and find out which variables drive smartphone prices the most:

Feature Selection:

1) Use techniques like correlation analysis, mutual information, or feature importance from tree-based models to identify the most relevant features.
Consider removing highly correlated features to avoid multicollinearity.
Model Selection:

2) Choose a regression model that suits your data and problem. Common choices include linear regression, decision tree regression, random forest regression, or gradient boosting algorithms.
Model Training:

3) Split your dataset into training and testing sets.
Train your chosen model on the training data.
Model Evaluation:

4) Evaluate the model's performance on the testing set using appropriate metrics (e.g., mean squared error, R-squared, etc.).
Analyze the residuals to check for any patterns or systematic errors.

5) Iterate and Refine hyperparameters etc.